In [1]:
import os
from pathlib import Path
from typing import Dict, Tuple, Optional

import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction

# ============================================================================
# 0. SETUP & LOAD DATA
# ============================================================================

os.chdir(r"E:\RPA–CRISPR-model")

# Load mutations table from previous notebook
mutations_df = pd.read_csv("data/mutations/tb_mdr_markers.csv")
mutations_df.loc[mutations_df["gene"] == "inhA_promoter", "genomic_position"] = 1674187
# Load extracted genomic regions
region_records = {
    "rpoB": SeqIO.read("data/sequences/rpoB_region.fasta", "fasta"),
    "katG": SeqIO.read("data/sequences/katG_region.fasta", "fasta"),
    "inhA_promoter": SeqIO.read("data/sequences/inhA_promoter.fasta", "fasta"),
}

regions = {name: str(rec.seq).upper() for name, rec in region_records.items()}

print("=" * 80)
print("LOADED MUTATIONS")
print("=" * 80)
print(mutations_df)
print("\n" + "=" * 80)
print("LOADED REGIONS")
print("=" * 80)
print("Correction appliquée pour inhA_promoter : position mise à jour vers 1674187")
print(mutations_df[mutations_df["gene"] == "inhA_promoter"])
for name, seq in regions.items():
    print(f"  {name:20s} : {len(seq):5d} bp")


LOADED MUTATIONS
            gene  genomic_position  wt_codon  mut_codon wt_aa mut_aa  \
0           rpoB            761155       NaN        NaN     S      L   
1           katG           2155168       NaN        NaN     S      T   
2  inhA_promoter           1674187       NaN        NaN   NaN    NaN   

  mutation_type drug         reference  \
0    SNP coding  RIF  PMCID:PMC9225881   
1    SNP coding  INH  PMCID:PMC9225881   
2  promoter SNP  INH  PMCID:PMC9225881   

                                                note  
0  rpoB codon 450 in M. tuberculosis (often repor...  
1  katG S315T; katG is on the reverse strand, so ...  
2  fabG1-inhA promoter C-15T; verify mapping in t...  

LOADED REGIONS
Correction appliquée pour inhA_promoter : position mise à jour vers 1674187
            gene  genomic_position  wt_codon  mut_codon wt_aa mut_aa  \
2  inhA_promoter           1674187       NaN        NaN   NaN    NaN   

  mutation_type drug         reference  \
2  promoter SNP  INH  PMCI

In [2]:

"""
Genomic coordinates on H37Rv (NC_000962.3, 1-based inclusive)
Sources:
  - rpoB: NCBI GenBank CCP43410.1, Mycobrowser Rv0667
  - katG: NCBI GenBank, Mycobrowser Rv1908c (note: reverse complement)
  - inhA: NCBI GenBank, Mycobrowser Rv1484 (promoter is upstream)
"""

# Genomic CDS coordinates (from NCBI / Mycobrowser)
GENE_COORDS_GENOMIC = {
    "rpoB": {
        "cds_start": 759807,   # 1-based, inclusive
        "cds_end": 763325,     # 1-based, inclusive
        "strand": "+",
        "n_codons": 1172,
    },
    "katG": {
        "cds_start": 2153889,
        "cds_end": 2156111,
        "strand": "-",  # reverse complement
        "n_codons": 740,
    },
    "inhA": {
        "cds_start": 1674202,
        "cds_end": 1675011,
        "strand": "+",
        "n_codons": 269,
    },
}

# Extracted region coordinates (from 01_tb_reference_and_mutations.ipynb)
REGION_COORDS_EXTRACTED = {
    "rpoB": {
        "region_start": 759807 - 200,   
        "region_end": 763325 + 200,     
        "strand": "+",
        "description": "rpoB coding region ± 200 bp",
    },
    "katG": {
        "region_start": 2153889 - 200,
        "region_end": 2156111 + 200,
        "strand": "+",  
        "description": "katG coding region ± 200 bp",
    },
    "inhA_promoter": {
        "region_start": 1674202 - 200,  
        "region_end": 1674202 + 50,     
        "strand": "+",
        "description": "inhA promoter + start codon region",
    },
}

print("Gene Coordinates on H37Rv (NC_000962.3):")
for gene, coords in GENE_COORDS_GENOMIC.items():
    print(f"  {gene:15s} : {coords['cds_start']:7d}..{coords['cds_end']:7d} "
          f"({coords['strand']}) | {coords['n_codons']} codons")

print("\nExtracted Region Coordinates:")
for region_key, coords in REGION_COORDS_EXTRACTED.items():
    actual_len = coords["region_end"] - coords["region_start"] + 1
    print(f"  {region_key:20s} : {coords['region_start']:7d}..{coords['region_end']:7d} "
          f"| {actual_len} bp (in memory: {len(regions[region_key])} bp)")


Gene Coordinates on H37Rv (NC_000962.3):
  rpoB            :  759807.. 763325 (+) | 1172 codons
  katG            : 2153889..2156111 (-) | 740 codons
  inhA            : 1674202..1675011 (+) | 269 codons

Extracted Region Coordinates:
  rpoB                 :  759607.. 763525 | 3919 bp (in memory: 3919 bp)
  katG                 : 2153689..2156311 | 2623 bp (in memory: 2623 bp)
  inhA_promoter        : 1674002..1674252 | 251 bp (in memory: 251 bp)


In [3]:
# 2. MUTATION POSITION MAPPING: Genomic → Regional Sequence Index

"""
Map a genomic position (1-based) to a 0-based index in the extracted region.

Key insight:
    - Genomic positions are 1-based (standard genomic notation)
    - Python sequence indices are 0-based
    - Region was extracted as: genome[start-1:end] (0-based slice)
    - So: index_in_region = genomic_position - region_start
"""

def map_genomic_to_region(
    region_key: str,
    genomic_pos: int,
) -> Tuple[int, bool]:
    """
    Map genomic position on H37Rv (1-based) to 0-based index in region sequence.
    
    Args:
        region_key: "rpoB", "katG", or "inhA_promoter"
        genomic_pos: 1-based position on H37Rv genome
        
    Returns:
        (idx_0based, in_bounds) : index in region or raises ValueError
        
    Raises:
        ValueError if genomic_pos is outside region bounds
    """
    coords = REGION_COORDS_EXTRACTED[region_key]
    start = coords["region_start"]
    end = coords["region_end"]
    
    if not (start <= genomic_pos <= end):
        raise ValueError(
            f"Genomic position {genomic_pos} outside region {region_key} "
            f"[{start}..{end}]"
        )
    
    
    idx_in_region = genomic_pos - start
    
    return idx_in_region


# Check mapping for each mutation
print("=" * 80)
print("MUTATION POSITION MAPPING (Genomic → Regional)")
print("=" * 80)

for _, row in mutations_df.iterrows():
    gene = row["gene"]
    gpos = int(row["genomic_position"])
    
    # Determine region key
    if gene == "inhA_promoter":
        region_key = "inhA_promoter"
    else:
        region_key = gene
    
    try:
        idx = map_genomic_to_region(region_key, gpos)
        region_len = len(regions[region_key])
        print(f"✓ {gene:20s} @ genomic {gpos:7d} → "
              f"region index {idx:5d} / {region_len:5d} bp")
    except ValueError as e:
        print(f"✗ {gene:20s} @ genomic {gpos:7d} → ERROR: {e}")


MUTATION POSITION MAPPING (Genomic → Regional)
✓ rpoB                 @ genomic  761155 → region index  1548 /  3919 bp
✓ katG                 @ genomic 2155168 → region index  1479 /  2623 bp
✓ inhA_promoter        @ genomic 1674187 → region index   185 /   251 bp


In [4]:
# 3. RPA AMPLICON DESIGN

"""
Propose an RPA amplicon around a mutation with multiple constraints:

Constraints (biologically justified):
    1. Length: 100–180 bp (RPA typically works best ~120–140 bp)
    2. GC%: 40–65% (avoid extreme secondary structures)
    3. SNP margin: ≥ 20 bp from amplicon edges (sensitivity & specificity)
    4. SNP interior: mutation must be strictly inside amplicon
    
Strategy:
    1. Try to center mutation in amplicon
    2. If centered window doesn't satisfy constraints, do local search
    3. Fallback: use closest feasible window if no perfect solution exists
    
References:
    - RPA: ~120–140 bp optimal length (Piepenburg et al. 2006, NAR)
    - GC constraints: standard for PCR/RPA primer design
    - SNP positioning: empirically ~30–50% from 5' end (detection sensitivity)
"""

def propose_rpa_amplicon(
    region_seq: str,
    mutation_idx_0based: int,
    target_len: int = 140,
    min_len: int = 100,
    max_len: int = 180,
    gc_min: float = 0.40,
    gc_max: float = 0.65,
    min_margin: int = 20,
) -> Dict:
    """
    Propose a simple RPA amplicon around a mutation.
    
    Args:
        region_seq: extracted region sequence (uppercase)
        mutation_idx_0based: 0-based index of SNP in region
        target_len: ideal amplicon length (default 140 bp)
        min_len, max_len: length bounds
        gc_min, gc_max: GC% bounds
        min_margin: minimum distance from SNP to amplicon edge (bp)
        
    Returns:
        dict with keys:
            - amp_start, amp_end: 0-based slice indices [start, end)
            - length: bp
            - gc: fraction (0–1)
            - snp_offset: 0-based offset of SNP from amp_start
            - sequence: actual nucleotide sequence
            - note: any warnings/fallbacks
    """
    n = len(region_seq)
    
    if mutation_idx_0based < min_margin or mutation_idx_0based >= n - min_margin:
        raise ValueError(
            f"Mutation at index {mutation_idx_0based} cannot satisfy margin "
            f"in region of length {n}"
        )
    
    # First attempt: center the mutation
    half = target_len // 2
    start = max(0, mutation_idx_0based - half)
    end = min(n, start + target_len)
   
    if end - start < target_len:
        start = max(0, end - target_len)
    
    seq = region_seq[start:end]
    gc = gc_fraction(seq)
    
    # Check if first attempt satisfies all constraints
    length = len(seq)
    snp_offset = mutation_idx_0based - start
    
    if (min_len <= length <= max_len and 
        gc_min <= gc <= gc_max and
        min_margin <= snp_offset <= length - min_margin):
        # Good!
        return {
            "amp_start": start,
            "amp_end": end,
            "length": length,
            "gc": gc,
            "snp_offset": snp_offset,
            "sequence": seq,
            "note": "Centered design satisfied all constraints.",
        }
    
    # If not satisfied, do a brute-force local search
    best = None
    search_range = range(min_len, min(max_len + 1, n))
    
    for L in search_range:
        # Minimum start where SNP has min_margin before it
        start_min = max(0, mutation_idx_0based - (L - min_margin - 1))
        # Maximum start where SNP has min_margin after it
        start_max = min(mutation_idx_0based - min_margin, n - L)
        
        if start_min > start_max:
            continue
        
        # Sample a few starts in this range (not all, for efficiency)
        starts_to_try = [start_min]
        if start_min < start_max:
            starts_to_try.append((start_min + start_max) // 2)
        if start_max not in starts_to_try:
            starts_to_try.append(start_max)
        
        for s in starts_to_try:
            e = s + L
            if not (0 <= s < e <= n):
                continue
            
            subseq = region_seq[s:e]
            gc = gc_fraction(subseq)
            
            if not (gc_min <= gc <= gc_max):
                continue
            
            snp_off = mutation_idx_0based - s
            
            if not (min_margin <= snp_off <= L - min_margin):
                continue
            
            best = {
                "amp_start": s,
                "amp_end": e,
                "length": L,
                "gc": gc,
                "snp_offset": snp_off,
                "sequence": subseq,
                "note": "Search-optimized design.",
            }
            return best  # Return first valid found
    
    # Fallback: return centered design with warning
    return {
        "amp_start": start,
        "amp_end": end,
        "length": len(seq),
        "gc": gc,
        "snp_offset": snp_offset,
        "sequence": seq,
        "note": (
            f"WARNING: Centered design does not satisfy all constraints. "
            f"GC={gc*100:.1f}%, length={length} bp. Proceeding with caution."
        ),
    }


# Test on one mutation as sanity check
print("=" * 80)
print("AMPLICON DESIGN TEST (First Mutation)")
print("=" * 80)

if len(mutations_df) > 0:
    row = mutations_df.iloc[0]
    gene = row["gene"]
    gpos = int(row["genomic_position"])
    region_key = gene if gene != "inhA_promoter" else "inhA_promoter"
    
    try:
        idx = map_genomic_to_region(region_key, gpos)
        amp = propose_rpa_amplicon(regions[region_key], idx)
        
        print(f"Gene: {gene}")
        print(f"  Genomic position: {gpos}")
        print(f"  Region index: {idx}")
        print(f"  Amplicon: [{amp['amp_start']}:{amp['amp_end']}] ({amp['length']} bp)")
        print(f"  GC%: {amp['gc']*100:.1f}%")
        print(f"  SNP offset in amplicon: {amp['snp_offset']} bp")
        print(f"  Note: {amp['note']}")
        print(f"  Sequence (first 60bp): {amp['sequence'][:60]}...")
    except Exception as e:
        print(f"ERROR: {e}")


AMPLICON DESIGN TEST (First Mutation)
Gene: rpoB
  Genomic position: 761155
  Region index: 1548
  Amplicon: [1478:1618] (140 bp)
  GC%: 68.3%
  SNP offset in amplicon: 70 bp
  Note: WARNING: Centered design does not satisfy all constraints. GC=68.3%, length=140 bp. Proceeding with caution.
  Sequence (first 60bp): GAGCTATCCGCGGTGACACTGGCTAACTGGGCCGCCAAGACCGGCAACCTGTTGCGCGAC...


In [5]:
# 4. DESIGN ALL AMPLICONS

print("=" * 80)
print("DESIGNING AMPLICONS FOR ALL MUTATIONS")
print("=" * 80)

designed = []

for idx_row, row in mutations_df.iterrows():
    gene = row["gene"]
    gpos = int(row["genomic_position"])
    drug = row["drug"]
    mut_type = row["mutation_type"]
    
    region_key = gene if gene != "inhA_promoter" else "inhA_promoter"
    region_seq = regions[region_key]
    
    try:
        # Map genomic position to regional index
        mut_idx = map_genomic_to_region(region_key, gpos)
        
        # Design amplicon
        amp = propose_rpa_amplicon(region_seq, mut_idx)
        
        designed.append({
            "gene": gene,
            "mutation_type": mut_type,
            "drug": drug,
            "genomic_position": gpos,
            "region_key": region_key,
            "region_length_bp": len(region_seq),
            "amplicon_start_0based": amp["amp_start"],
            "amplicon_end_0based": amp["amp_end"],
            "amplicon_length_bp": amp["length"],
            "amplicon_gc_percent": round(amp["gc"] * 100, 1),
            "snp_offset_in_amplicon_bp": amp["snp_offset"],
            "amplicon_sequence": amp["sequence"],
            "design_note": amp["note"],
        })
        
        print(f"✓ {gene:20s} ({drug:3s}): {amp['length']:3d} bp, "
              f"GC={amp['gc']*100:5.1f}%, SNP @ {amp['snp_offset']:3d} bp")
        
    except Exception as e:
        print(f"✗ {gene:20s}: {e}")

designed_df = pd.DataFrame(designed)

print("\n" + "=" * 80)
print("DESIGNED AMPLICONS SUMMARY")
print("=" * 80)
print(designed_df[["gene", "drug", "amplicon_length_bp", "amplicon_gc_percent", 
                    "snp_offset_in_amplicon_bp"]].to_string(index=False))


DESIGNING AMPLICONS FOR ALL MUTATIONS
✓ rpoB                 (RIF): 140 bp, GC= 68.3%, SNP @  70 bp
✓ katG                 (INH): 140 bp, GC= 60.7%, SNP @  70 bp
✓ inhA_promoter        (INH): 140 bp, GC= 63.6%, SNP @  74 bp

DESIGNED AMPLICONS SUMMARY
         gene drug  amplicon_length_bp  amplicon_gc_percent  snp_offset_in_amplicon_bp
         rpoB  RIF                 140                 68.3                         70
         katG  INH                 140                 60.7                         70
inhA_promoter  INH                 140                 63.6                         74


In [6]:
# 5 Export results

output_dir = Path("data/simulated")
output_dir.mkdir(parents=True, exist_ok=True)

# Save full design table
designed_df.to_csv(output_dir / "rpa_amplicons_designed.csv", index=False)
print(f"\n✓ Saved: data/simulated/rpa_amplicons_designed.csv")

# Save amplicon sequences in FASTA format (for synthesis, validation)
fasta_out = output_dir / "rpa_amplicons.fasta"
with open(fasta_out, "w") as f:
    for idx, row in designed_df.iterrows():
        fasta_id = (f"{row['gene']}_{row['drug']}_amp_{idx}")
        description = (f"len={row['amplicon_length_bp']}bp "
                      f"gc={row['amplicon_gc_percent']}% "
                      f"snp_offset={row['snp_offset_in_amplicon_bp']}bp")
        f.write(f">{fasta_id} {description}\n")
        # Write sequence in 80bp lines
        seq = row["amplicon_sequence"]
        for i in range(0, len(seq), 80):
            f.write(seq[i:i+80] + "\n")

print(f"✓ Saved: data/simulated/rpa_amplicons.fasta")

print("\n" + "=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)
print(f"Total mutations designed: {len(designed_df)}")
print(f"Mean amplicon length: {designed_df['amplicon_length_bp'].mean():.1f} bp")
print(f"Mean GC%: {designed_df['amplicon_gc_percent'].mean():.1f}%")
print(f"Drugs covered: {designed_df['drug'].unique()}")



✓ Saved: data/simulated/rpa_amplicons_designed.csv
✓ Saved: data/simulated/rpa_amplicons.fasta

SUMMARY STATISTICS
Total mutations designed: 3
Mean amplicon length: 140.0 bp
Mean GC%: 64.2%
Drugs covered: ['RIF' 'INH']


In [7]:
# 6. VALIDATION & QUALITY METRICS

"""
Quick sanity checks on the design.
"""

print("=" * 80)
print("DESIGN VALIDATION")
print("=" * 80)

# Check 1: SNP positioning
print("\n1. SNP Positioning in Amplicons:")
for _, row in designed_df.iterrows():
    L = row["amplicon_length_bp"]
    offset = row["snp_offset_in_amplicon_bp"]
    pct_from_5prime = 100 * offset / L
    status = "✓" if 20 <= offset <= L - 20 else "⚠"
    print(f"  {status} {row['gene']:20s}: {offset:3d}/{L:3d} bp ({pct_from_5prime:5.1f}% from 5')")

# Check 2: GC% compliance
print("\n2. GC% Compliance (target 40–65%):")
for _, row in designed_df.iterrows():
    gc = row["amplicon_gc_percent"]
    status = "✓" if 40 <= gc <= 65 else "⚠"
    print(f"  {status} {row['gene']:20s}: {gc:5.1f}%")

# Check 3: Length compliance
print("\n3. Length Compliance (target 100–180 bp, optimal ~140 bp):")
for _, row in designed_df.iterrows():
    L = row["amplicon_length_bp"]
    status = "✓" if 100 <= L <= 180 else "⚠"
    dev = L - 140
    print(f"  {status} {row['gene']:20s}: {L:3d} bp (Δ {dev:+3d} from target)")

print("\n" + "=" * 80)


DESIGN VALIDATION

1. SNP Positioning in Amplicons:
  ✓ rpoB                :  70/140 bp ( 50.0% from 5')
  ✓ katG                :  70/140 bp ( 50.0% from 5')
  ✓ inhA_promoter       :  74/140 bp ( 52.9% from 5')

2. GC% Compliance (target 40–65%):
  ⚠ rpoB                :  68.3%
  ✓ katG                :  60.7%
  ✓ inhA_promoter       :  63.6%

3. Length Compliance (target 100–180 bp, optimal ~140 bp):
  ✓ rpoB                : 140 bp (Δ  +0 from target)
  ✓ katG                : 140 bp (Δ  +0 from target)
  ✓ inhA_promoter       : 140 bp (Δ  +0 from target)

